## Integração de dados

#### Masterlens, CASTLES

In [2]:
import pandas as pd
from astropy.coordinates import Angle

In [3]:
# dec: -27:52:25.7 ---> -27.873806 deg
def sexdeg_to_deg(coord):
    result = Angle(coord + " degrees").degree
    return result #*3600


In [4]:
# ra: 00h49m41.89s ---> 12.424542 deg
def hour_to_deg(coord):  # string coord
    result = Angle(coord).degree
    return result #*3600

In [5]:
# ra: 00:49:41.89 ---> 00h49m41.89s
def hour(coord):
    result = coord.replace(":", 'h', 1)
    result = result.replace(":", 'm', 1)
    result += 's'
    return result

In [6]:
# system table
def system():  # lens as a whole
    
    # importing castles data  -  100 lenses
    castles = pd.read_csv('Scraping/castles.csv')
    castles.drop(['zs', 'zl', 'ms (mag)', 'ml (mag)', 'dt (days)'], axis=1, inplace=True)
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour)
    castles['RA (J2000)'] = castles['RA (J2000)'].apply(hour_to_deg)
    castles['Dec (J2000)'] = castles['Dec (J2000)'].apply(sexdeg_to_deg)
    castles.columns=['Name', 'RA(deg)_c', 'Dec(deg)_c', 'E(B-V)_c', 'FGHz (mJy)_c', 'Nimages_c',
                     'size(arccsec)_c', 'sigma (km/s)_c']
    castles = castles.set_index('Name')
    
    
    # importing masterlens data  -  50 lenses
    dfs = []
    for i in range(1,51):
        
        # getting data from system file
        ml = pd.read_csv('Scraping/results/system_{}.csv'.format(i))

        # selecting important columns
        cols=[]
        for column in ['Name', 'Alternate Names', 'Discovery', 'Discovery Date', 'Lens Kind', 'Lens Grade',
                       'Description', 'N Images', 'Einstein_R', 'Einstein_R quality',
                       'Stellar_v_disp', 'Stellar_v_disp_err']:
            if column in list(ml.columns):
                cols.append(column)
        ml = ml[cols]

        # getting coords from coords file
        coords = pd.read_csv('Scraping/results/coordinates_{}.csv'.format(i))
        ml['Ra(deg)_ml'] = coords['RA [°]']
        ml['Dec(deg)_ml'] = coords['Dec [°]']

        dfs.append(ml)
     
    # joining all masterlens system + coords dfs
    all_ml = pd.concat(dfs, sort=False)
    all_ml = all_ml.set_index('Name')

    # joining castles + ml data
    result = pd.merge(castles, all_ml, how ='outer', on='Name')  

    return result
        


In [7]:
df = system()
df

,RA(deg)_c,Dec(deg)_c,E(B-V)_c,FGHz (mJy)_c,Nimages_c,size(arccsec)_c,sigma (km/s)_c,Discovery,Discovery Date,Lens Kind,Lens Grade,Description,N Images,Einstein_R,Einstein_R quality,Stellar_v_disp,Stellar_v_disp_err,Ra(deg)_ml,Dec(deg)_ml,Alternate Names
Name,,,,,,,,,,,,,,,,,,,,
Q0047-2808,12.424542,-27.873806,0.016,NaN,4ER,2.70,229±15,Serendipitous,1996-01-01,GAL-QSO,A,Bright Einstein ring-like feature. The source ...,4.0,1.340,NaN,219.0,12.0,12.424747,-27.874020,ER 0047-2808
HE0047-1756,12.615958,-17.669111,0.022,NaN,2,1.44,NaN,HE survey,2004-05-01,GAL-QSO,A,The newly discovered double QSO HE 0047−1756 ...,2.0,0.751,SIE model,NaN,NaN,12.615500,-17.668890,NaN
HST01247+0352,21.185000,3.866667,0.029,NaN,2,2.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HST01248+0351,21.190000,3.851667,0.029,NaN,2,0.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0128+437,22.805854,43.970317,0.082,F5=48,4,0.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PMNJ0134-0931,23.648625,-9.517472,0.031,F5=529,5R,0.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q0142-100,26.318750,-9.754722,0.031,F5~1,2,2.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
QJ0158-4325,29.672667,-43.417833,0.015,F8<0.2,2,1.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0218+357,35.272846,35.937161,0.068,F5=1209,2ER,0.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# comparing 'alternate names' with 'name'      partial merge?

for index, row in df.iterrows():
    if type(row['Alternate Names']) == str:
        row['Alternate Names'] = row['Alternate Names'].split(', ')  # list with alternate names
        for i in row['Alternate Names']:
            if index == i:
                print('problem')


In [25]:
# Scraping Joao
# lens objects  -  227
castles2 = pd.read_csv('CastelLensData.csv')
castles2 = castles2.set_index('lens_names')

HE_c = castles2.loc[ 'HE0435-1223' , : ]
HE_c

,ra,ra_err,dec,dec_err,description
lens_names,,,,,
HE0435-1223,4.63747,0.000,-12.2873,0.000,deflected
HE0435-1223,3.16147,0.003,-11.7343,0.003,deflected
HE0435-1223,2.17047,0.003,-12.8903,0.005,deflected
HE0435-1223,3.69847,0.003,-13.9013,0.003,deflected
HE0435-1223,3.47247,0.003,-12.8603,0.003,galaxy


In [31]:
# Object table
HE_ml = pd.read_csv('Scraping/results/redshift_28.csv')
HE_ml.drop('Unnamed: 1', axis=1, inplace=True)
HE_ml.rename(columns={"Unnamed: 0": "Description"})

,Description,Mag,No.,z
0,Lens,NaN,1,0.460
1,Source,18.3,1,1.689
2,Source,18.2,2,1.689
3,Source,18.2,3,1.689
4,Source,18.8,4,1.689
